### Detects the onset of sound in the mix and in each part.

**2024/06/25 - The current shifts_pre_mix.txt files don't differ by a fixed amount from the shifts.txt files, so I think this script was unused. It's currently set to shift each part using the first sound of each part that exceeds a fixed threshold. I don't think that worked. I ended up doing it all by ear. Even though that seems like a lot of work compared to shifting each part by the same amount.**

**Changes the shift amounts of each part so that they line up with the mix.**

- mix_shift = mix start - offset of part with current shift 0
- add mix_shift to each part's shift (the parts are already shifted to line up) 

Example (in seconds)

- mix entrance at 1
- current shifts (made by hand) are
- - bass: 0
- - middle: 2
- - top: 4
- entrances are
- - bass: 2
- - middle: 4.1
- - top: 5.9

mix_shift = mix entrance - bass entrance = -1

new shifts are

- bass: 1
- middle: 3
- top: 5

**Compares the handmade shifts to the ones this script makes.**

- (middle entrance - bass entrance) - middle shift = 2.1 - 2 = 0.1
- (top entrance - bass entrance) - top shift = 3.9 - 4 = -0.1


In [ ]:
import librosa
from scipy import optimize
from scipy import interpolate
from scipy.signal import butter, argrelmin, argrelmax, argrelextrema
from scipy.fft import fft, ifft, rfft, irfft
import random
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
shift_dir = '/Akamai/voice/data/ground-estimate/'
audio_dir = '/Akamai/voice/data/'

In [ ]:
def load_file (song_part_path):
    y, sr = librosa.load(song_part_path, sr=None)
    return (y, sr)


In [ ]:
# find first sound (mic turning on is > 0.005 but < 0.01; sound is > 0.01)
def first_sound(y, sr, thresh = 0.01):
    return np.argmax(y > thresh)/sr

In [ ]:
working_song = ""

part_name = ['bass', 'middle', 'top']

suffix_part = {'1M': 0, 's)': 0, 
               '2M': 1, 'e)': 1, 
               '3M': 2, 'p)': 2,
               'x)': 3}

threshold_overrides = {
    "Manana": 0.02,
    "Ak'a Si Rekisho": 0.001
}

#collections = ["Teach Yourself Megrelian Songs", "Teach Yourself Gurian Songs", "Scherbaum Mshavanadze"]
collections = ["Teach Yourself Gurian Songs", "Teach Yourself Megrelian Songs"]
for coll in collections:
    for song in os.listdir(f"{audio_dir}{coll}"):
        if song != working_song and working_song != "":
            continue
        if os.path.isdir(f"{audio_dir}{coll}/{song}"):
            if song != 'mp3' and song != '.ipynb_checkpoints':
                
                # get audio file paths
                parts = []
                for part in os.listdir(f"{audio_dir}{coll}/{song}"):
                    if part[-4:] == '.wav':
                        part_num = 3
                        if part[-6:-4] in suffix_part:
                            part_num = suffix_part[part[-6:-4]]
                        parts.append((part_num, f"{audio_dir}{coll}/{song}/{part}"))
                
                first_sounds = [0, 0, 0, 0]
                for part_num, path in parts:
                    y, sr = load_file(path)
                    if song in threshold_overrides:
                        first_sounds[part_num] = first_sound(y, sr, threshold_overrides[song])
                    else:
                        first_sounds[part_num] = first_sound(y, sr)
                    
                
                # get handmade shifts
                shiftLists = []
                unshifted_part = 0
                part = 0
                shift_path = f"{shift_dir}{coll}/{song}/shifts_pre_mix.txt"
                with open(shift_path) as shift_file:
                    lines = shift_file.readlines()
                    for line in lines:
                        if line[0] == '#':
                            break
                        shiftList = list(map(float, line.split(' ')))
                        if shiftList[2] == 0.0:
                            unshifted_part = part
                        shiftLists.append(shiftList)
                        part += 1
                
                print(song)
                # compare handmade shifts to differences between entrances
                '''
                print(" Part, First sound, Handmade shifts, Entrance offsets, Difference:")
                for i in range(3):
                    hs = shiftLists[i][2]
                    eo = first_sounds[unshifted_part] - first_sounds[i]
                    print(f" {part_name[i]}, {first_sounds[i]}, {hs}, {eo}, {eo-hs}")
                '''       
                # shift shifts
                mix_shift = first_sounds[3] - first_sounds[unshifted_part]
                #print(f" Mix first sound: {first_sounds[3]}, mix shift: {mix_shift}")
                #print("Shifts before:",shiftLists)
                for i in range(3):
                    # using handmade: 
                    #shiftLists[i][2] = shiftLists[i][2] + mix_shift
                    shiftLists[i][2] = first_sounds[3] - first_sounds[i] 
                
                #print("Shifts after:",shiftLists)
                dest = f"{shift_dir}{coll}/{song}/shifts.txt"
                np.savetxt(dest, shiftLists, fmt="%.2f")
                dest = f"{shift_dir}{coll}/{song}/first_sounds.txt"
                np.savetxt(dest, np.array([first_sounds]), fmt="%.2f")
                

In [ ]:
li = [1, 2, 3]
lis = [li]
lit = [[0, 1, 2]]
lit[0] = lis[0]
lit